In [2]:
from selenium import webdriver
from urllib import parse
import bs4
import re
import time

## Connect database

In [83]:
dblocation = '/Users/graceli/Desktop/SBR automation/Company Database.db'
import pandas as pd
import sqlite3
conn = sqlite3.connect(dblocation)

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
AZ = pd.read_sql_query("SELECT CompanyName from CompanyData WHERE State='AZ'", conn)

In [84]:
AZ['CompanyName']

0                          Abitibi Consolidated Sales
1                                      AdobeAir, Inc.
2                          Aerotech Laboratories Inc.
3                         Anthony Manufacturing Corp.
4                 Arch Aluminum & Glass Company of AZ
5                       Arizona Precision Sheet Metal
6                           Arizona Surgical Hospital
7                                  Arrowhead Hospital
8                                          Asarco LLC
9                  Atrium Door & Window Company of AZ
10                                          Avnet Inc
11                      Banner Baywood Heart Hospital
12                      Banner Baywood Medical Center
13                       Banner Desert Medical Center
14                     Banner Estrella Medical Center
15                   Banner Good Samaritan Med Center
16                               Banner Mesa Hospital
17                  Banner Thunderbird Medical Center
18                      Bard

## Search function

In [110]:
def AZbusiness(business):
    driver = webdriver.Chrome()
    url = 'https://ecorp.azcc.gov/EntitySearch/Index/'
    
    #initiate the webdriver and passin the link
    driver.get(url)
    companySearch = driver.find_element_by_id("quickSearch_BusinessName")
    companySearch.send_keys(business)
    companySearch.send_keys(u'\ue007')
    time.sleep(5)
    html = driver.page_source
    soup = bs4.BeautifulSoup(html,'html.parser')
    if soup.find('tbody') == None:
        #print('no')
        driver.quit()
        return [' ']*7
    url = soup.find('a',{'class':'BlueLink'})['href']
    newurl = 'https://ecorp.azcc.gov/' + url
    driver.get(newurl)
    #print('yes')
    html = driver.page_source
    soup = bs4.BeautifulSoup(html,'html.parser')
    try:
        regName = soup.find(string="Entity Name:").find_next('div').text.strip()
        regNumber = soup.find(string="Entity ID:").find_next('div').text.strip()
        RegDate = soup.find(string="Approval Date:").find_next('div').text.strip()
        Jurisdiction = soup.find(string="Domicile State:").find_next('div').text.strip()
        entType = soup.find(string="Entity Type:").find_next('div').text.strip()
        status = soup.find(string="Entity Status:").find_next('div').text.strip()
        address = ' ' 
    except: 
        driver.quit()
        return [' ']*7
    driver.quit()
    
    return [regNumber, regName, RegDate, Jurisdiction, entType, status, address]

In [82]:
#driver = webdriver.Chrome()
#business = 'abc comp'
#AZbusiness(business, driver)

['01133350',
 'A.B.C. COMPLETE MOBILE HOME SERVICE, INC.',
 '',
 '',
 'Unidentified',
 'Inactive',
 ' ']

## Save results

In [90]:
AZ_result = []

In [101]:
for business in AZ['CompanyName'][42:]:
    searchResult = AZbusiness(business, driver)
    AZ_result.append({'CompanyName':business,'regNumber':searchResult[0], 'regName':searchResult[1],
                                    'RegDate':searchResult[2], 'Jurisdiction':searchResult[3],
                                     'entType':searchResult[4],
                                     'status': searchResult[5],'address':searchResult[6]})

In [102]:
AZ_results = pd.DataFrame(AZ_result)[['CompanyName','regName','regNumber','RegDate', 'Jurisdiction','entType','status','address']]

In [104]:
AZ_results

,CompanyName,regName,regNumber,RegDate,Jurisdiction,entType,status,address
0,Abitibi Consolidated Sales,ABITIBI CONSOLIDATED SALES CORPORATION,F08366640,4/1/1998,DE,Foreign For-Profit (Business) Corporation,Inactive,
1,"AdobeAir, Inc.","ADOBE AIR CONDITIONING & HEATING, LLC",L15138362,3/25/2009,AZ,Domestic LLC,Active,
2,Aerotech Laboratories Inc.,"AEROTECH LABORATORIES, INC.",06013231,6/20/2018,AZ,Domestic For-Profit (Business) Corporation,Active,
3,Anthony Manufacturing Corp.,ANTHONY MANUFACTURING CORP.,F00495107,5/7/1992,CA,Foreign For-Profit (Business) Corporation,Inactive,
4,Arch Aluminum & Glass Company of AZ,,,,,,,
5,Arizona Precision Sheet Metal,"ARIZONA PRECISION SHEET METAL, INC.",00718981,10/11/1967,AZ,Domestic For-Profit (Business) Corporation,Active,
6,Arizona Surgical Hospital,,,,,,,
7,Arrowhead Hospital,"ARROWHEAD HOSPITALITY GROUP, L.L.C.",L07428844,2/21/1995,AZ,Domestic LLC,Active,
8,Asarco LLC,"ASARCO ARIZONA, INC.",F07655020,12/29/1995,NY,Foreign For-Profit (Business) Corporation,Inactive,
9,Atrium Door & Window Company of AZ,,,,,,,


In [108]:
AZ_results.to_csv('AZ_results.csv',index=False)